### Preparar ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [2]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
from datetime import datetime
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [3]:
import pandas as pd
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')
bucket='results-and-tables-graphdata'

In [4]:
import os
import pandas as pd
import zipfile
import shutil
from io import StringIO
import io

### Declaración funciones para Impo-Expo s3

In [5]:
def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=";" , line_terminator='\n')
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print("Writing {} records to {}".format(len(dataframe), filename))
    print("S3")
    print(dataframe.shape)
    

In [6]:
def write_dataframe_to_excel_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a excel on S3 """
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            dataframe.to_excel(writer,index=False)
        data = output.getvalue()

    s3 = boto3.resource('s3')
    s3.Bucket(bucket).put_object(Key=filename, Body=data)
    

In [7]:
def import_dataframe_csv_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=";")
    return df

In [8]:
def import_dataframe_excel_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    obj = client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
    return df

In [9]:
def write_file_to_s3(bucket,file_local,file_s3):
    s3.Bucket(bucket).upload_file(file_local,file_s3)

### Importar tabla de Athena

#### Realizando prueba

In [10]:
#toda la base
min_distancia = 1 #km
min_pings_dia = 10
dfs_indicadores = []

tabla_usuarios_distancia =  f'historico_usuarios'    
query_filtro = (f"SELECT t.year, "
                    "t.month, "
                    "t.day, "
                    "t.iso_country_code, "
                    "COUNT(*) as n_users, "
                    f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                    f"FROM ("
                        "SELECT * "
                        f"FROM {tabla_usuarios_distancia} "
                        f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia})) AS t "
                    "GROUP BY t.year, t.month, t.day, t.iso_country_code")

df = cursor.execute(query_filtro).as_pandas()
df['%MasDe1km'] = df['n_users_mas1km'] / df['n_users']*100
dfs_indicadores.append(df)
df_by_day2athena = pd.concat(dfs_indicadores)
df_by_day2athena.head()

,year,month,day,iso_country_code,n_users,n_users_mas1km,%MasDe1km
0,2020,2,1,VE,42445,26735,62.987395
1,2020,2,14,GY,3827,2535,66.239875
2,2020,2,19,SV,15307,10232,66.845234
3,2020,2,26,PY,15346,10948,71.341066
4,2020,2,24,HN,21707,14129,65.089602


#### Periodo de analisis

In [11]:
### Declarando condiciones

# periodo particular
mes_inicio=2
dia_inicio=1

mes_final=4
dia_final= 30

# condiciones de filtro
min_distancia = 1 #km
min_pings_dia = 10

# matriz vacia para rellenar informacion
dfs_indicadores = []

### Generando tabla
if mes_inicio==mes_final:
    tabla_usuarios_distancia =  f'historico_usuarios'    
    query_filtro = (f"SELECT t.year, "
                        "t.month, "
                        "t.day, "
                        "t.iso_country_code, "
                        "t.id_provincia, "
                        "COUNT(*) as n_users, "
                        f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                        f"FROM ("
                            "SELECT * "
                            f"FROM {tabla_usuarios_distancia} "
                            f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia} AND ({tabla_usuarios_distancia}.day >= {dia_inicio} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_inicio}) AND ({tabla_usuarios_distancia}.day <= {dia_final} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_final}))) AS t "
                        "GROUP BY t.year, t.month, t.day, t.iso_country_code, t.id_provincia")
else:
    tabla_usuarios_distancia =  f'historico_usuarios'    
    query_filtro = (f"SELECT t.year, "
                        "t.month, "
                        "t.day, "
                        "t.iso_country_code, "
                        "t.id_provincia, "
                        "COUNT(*) as n_users, "
                        f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                        f"FROM ("
                            "SELECT * "
                            f"FROM {tabla_usuarios_distancia} "
                            f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia} AND (({tabla_usuarios_distancia}.day >= {dia_inicio} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_inicio}) OR ({tabla_usuarios_distancia}.day <= {dia_final} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_final})))) AS t "
                        "GROUP BY t.year, t.month, t.day, t.iso_country_code, t.id_provincia")
    
df = cursor.execute(query_filtro).as_pandas()
df['%MasDe1km'] = df['n_users_mas1km'] / df['n_users']*100
dfs_indicadores.append(df)

###operaciones
df_by_day2athena = pd.concat(dfs_indicadores)
df_by_day2at=df_by_day2athena
df_by_day2at=pd.DataFrame(df_by_day2at)
df_by_day2at=df_by_day2at.dropna(subset=["id_provincia"])
df_by_day2at['id_provincia']=df_by_day2at['id_provincia'].astype('int')
df_by_day2at['month']=df_by_day2at['month'].astype('int')
df_by_day2at['day']=df_by_day2at['day'].astype('int')
df_by_day2at["day"]=df_by_day2at["day"].map("{:02}".format)
df_by_day2at["key4"]=df_by_day2at['month'].astype(str)+ df_by_day2at["day"].astype(str)
df_by_day2at['key4']=df_by_day2at['key4'].astype('int')
df_by_day2at=df_by_day2at.rename(columns={'month':'Mes','day':'Dia','%MasDe1km':'%MasDe1km_new','n_users_mas1km':'MasDe1km'})
df_by_day2at=df_by_day2at.loc[:,['Mes','Dia','key4','id_provincia','MasDe1km','n_users','%MasDe1km_new','iso_country_code']]
df_by_day2at['Dia']=df_by_day2at['Dia'].astype('int')

###revisar columnas de informacion
df_by_day2at.head()

,Mes,Dia,key4,id_provincia,MasDe1km,n_users,%MasDe1km_new,iso_country_code
0,2,21,221,32,540,836,64.593301,DO
1,2,21,221,29,168,236,71.186441,DO
2,2,20,220,2,100,149,67.114094,DO
3,2,17,217,3,300,496,60.483871,DO
4,2,13,213,16,559,878,63.667426,DO


In [12]:
df_by_day2at.shape

(22682, 8)

#### Segundo periodo de analisis - Activar si el periodo es discontinuo o se actualiza constantemente

In [13]:
### condiciones para empalmar

## periodo particular
mes_inicio=9
dia_inicio=1

mes_final=9
dia_final= 30

## condiciones de filtro
min_distancia = 1 #km
min_pings_dia = 10

## matriz vacia para rellenar informacion
dfs_indicadores = []


### Generando tabla
if mes_inicio==mes_final:
    tabla_usuarios_distancia =  f'historico_usuarios'    
    query_filtro = (f"SELECT t.year, "
                        "t.month, "
                        "t.day, "
                        "t.iso_country_code, "
                        "t.id_provincia, "
                        "COUNT(*) as n_users, "
                        f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                        f"FROM ("
                            "SELECT * "
                            f"FROM {tabla_usuarios_distancia} "
                            f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia} AND ({tabla_usuarios_distancia}.day >= {dia_inicio} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_inicio}) AND ({tabla_usuarios_distancia}.day <= {dia_final} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_final}))) AS t "
                        "GROUP BY t.year, t.month, t.day, t.iso_country_code, t.id_provincia")
else:
    tabla_usuarios_distancia =  f'historico_usuarios'    
    query_filtro = (f"SELECT t.year, "
                        "t.month, "
                        "t.day, "
                        "t.iso_country_code, "
                        "t.id_provincia, "
                        "COUNT(*) as n_users, "
                        f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                        f"FROM ("
                            "SELECT * "
                            f"FROM {tabla_usuarios_distancia} "
                            f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia} AND (({tabla_usuarios_distancia}.day >= {dia_inicio} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_inicio}) OR ({tabla_usuarios_distancia}.day <= {dia_final} "
                   f"AND {tabla_usuarios_distancia}.month = {mes_final})))) AS t "
                        "GROUP BY t.year, t.month, t.day, t.iso_country_code, t.id_provincia")
    
df = cursor.execute(query_filtro).as_pandas()
df['%MasDe1km'] = df['n_users_mas1km'] / df['n_users']*100
dfs_indicadores.append(df)


###operaciones
df_by_dayempalme = pd.concat(dfs_indicadores)
df_by_day_empalme=df_by_dayempalme
df_by_day_empalme=pd.DataFrame(df_by_day_empalme)
df_by_day_empalme=df_by_day_empalme.dropna(subset=["id_provincia"])
df_by_day_empalme['id_provincia']=df_by_day_empalme['id_provincia'].astype('int')
df_by_day_empalme['month']=df_by_day_empalme['month'].astype('int')
df_by_day_empalme['day']=df_by_day_empalme['day'].astype('int')
df_by_day_empalme["day"]=df_by_day_empalme["day"].map("{:02}".format)
df_by_day_empalme["key4"]=df_by_day_empalme['month'].astype(str)+ df_by_day_empalme["day"].astype(str)
df_by_day_empalme['key4']=df_by_day_empalme['key4'].astype('int')
df_by_day_empalme=df_by_day_empalme.rename(columns={'month':'Mes','day':'Dia','%MasDe1km':'%MasDe1km_new','n_users_mas1km':'MasDe1km'})
df_by_day_empalme=df_by_day_empalme.loc[:,['Mes','Dia','key4','id_provincia','MasDe1km','n_users','%MasDe1km_new','iso_country_code']]
df_by_day_empalme['Dia']=df_by_day_empalme['Dia'].astype('int')


###juntando base de datos
df_by_day2at = df_by_day2at.append(df_by_day_empalme, ignore_index=True)

###revisar columnas de informacion
df_by_day2at.head()


,Mes,Dia,key4,id_provincia,MasDe1km,n_users,%MasDe1km_new,iso_country_code
0,2,21,221,32,540,836,64.593301,DO
1,2,21,221,29,168,236,71.186441,DO
2,2,20,220,2,100,149,67.114094,DO
3,2,17,217,3,300,496,60.483871,DO
4,2,13,213,16,559,878,63.667426,DO


#### Ajustes finales

In [14]:
df_by_day2at=df_by_day2at.sort_values(by=['Mes','Dia','iso_country_code','id_provincia'])
df_by_day2at.head()

,Mes,Dia,key4,id_provincia,MasDe1km,n_users,%MasDe1km_new,iso_country_code
2272,2,1,201,1,94936,136632,69.482991,AR
771,2,1,201,2,17828,27366,65.146532,AR
16693,2,1,201,3,875,1370,63.868613,AR
13223,2,1,201,4,2595,3890,66.709512,AR
10518,2,1,201,5,2610,3774,69.157393,AR


In [15]:
df_by_day2at.shape

(34193, 8)

### Indices Nacionales

In [16]:
paises = ['DO','SV','CR','BO','CL','EC','AR','VE','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','BR','MX'] 
#paises = ['CL']
for pais in paises:
    print(f"Pais: {pais}")
    
    df_by_day=df_by_day2at[df_by_day2at['iso_country_code']==pais]
    df_by_day=df_by_day.rename(columns={'iso_country_code':'country'})
    df_by_day=df_by_day.sort_values(by=['Mes','Dia'])
    
    ##computar indice suavizado
    
    ##Baseline    
    year="2020"
    df_by_day["fecha"] = year + "-" + df_by_day['Mes'].astype(int).astype(str) + "-" + df_by_day["Dia"].astype(int).astype(str)
    df_by_day['weekday'] = pd.to_datetime(df_by_day['fecha']).apply(lambda x: x.weekday())
    df_by_day["weekday2"]=0 
    df_by_day["weekday2"][df_by_day["weekday"]==6]="Domingo"       
    df_by_day["weekday2"][df_by_day["weekday"]==5]="Sabado" 
    df_by_day["weekday2"][(df_by_day["weekday"]!=5) & (df_by_day["weekday"]!=6)]="Semana" 
    df_by_day["key"]=df_by_day["weekday2"].astype(str)+df_by_day["country"].astype(str)
    reg=df_by_day["country"].unique()
    dia8=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==8)].groupby(["country"])["%MasDe1km_new"].sum()
    dia8=pd.DataFrame(dia8)
    dia8["fecha"]="Domingo"
    
    #dia8["country"]=reg
    dia8=dia8.reset_index()
    dia7=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==7)].groupby(["country"])["%MasDe1km_new"].sum()
    dia7=pd.DataFrame(dia7)
    dia7["fecha"]="Sabado"
    
    #dia7["country"]=reg
    dia7=dia7.reset_index()
    diasem=df_by_day[(df_by_day["Mes"]==2) & (df_by_day['Dia'].isin(['5', '6', '9', '10', '11']))].groupby(["country"])["%MasDe1km_new"].mean()
    diasem=pd.DataFrame(diasem)
    diasem["fecha"]="Semana"
    
    #diasem["country"]=reg
    diasem=diasem.reset_index()
    match=diasem.append(dia8)
    match=match.append(dia7)
    match["key"]=match["fecha"].astype(str)+match["country"].astype(str)
    match=match.rename(columns={"%MasDe1km_new": "Baseline"})
    match=match.loc[:,["key","Baseline"]]
    df_by_day=df_by_day.merge(match,how='left',on=['key'])
    
    ##Ratio
    df_by_day["ratio"]=df_by_day['%MasDe1km_new']/df_by_day['Baseline']
    
    ##por excepcion del día conflictivo
    df_by_day["ratio_lag1"]=df_by_day.groupby(["country"])["ratio"].shift(1)
    df_by_day["ratio_f1"]=df_by_day.groupby(["country"])["ratio"].shift(-1)
    
    ##Suavizado
    df_by_day["ratio_sua"]=(df_by_day["ratio"]+df_by_day["ratio_lag1"]+df_by_day["ratio_f1"])/3
    
    ##Indice
    df_by_day["FracMas1km"]=df_by_day["ratio_sua"]-1
    
    ##key para tabla pivot
    df_by_day["Dia"]=df_by_day["Dia"].astype(int)
    df_by_day["Dia"]=df_by_day.Dia.map("{:02}".format)
    df_by_day["key4"]=df_by_day['Mes'].astype(int).astype(str)+ df_by_day["Dia"].astype(str)
    
    ##genero dataframe para exportar
    df_by_day2=df_by_day.loc[:,["Mes", "Dia","key4", "country", "MasDe1km", "n_users",'%MasDe1km_new',"FracMas1km"]]
    
    ##tabla pivot para publicar
    table_piv = pd.pivot_table(df_by_day, values='FracMas1km', index=['country'],
                    columns=['key4'], aggfunc=np.sum)
    table_piv=table_piv[table_piv.columns[1:-1]]
    table_piv = table_piv*100
    table_piv = table_piv.round(2)
    table_piv.index.name="cod"
    
    ruta=f'graphdata-mobility-public/indicador_por_pais_nacional/{pais}_nacion_analisis.csv'
    write_dataframe_to_csv_on_s3(df_by_day2, 'iadbprod-csd-hub-analyticaldata', ruta)
    
    ruta2=f'graphdata-mobility-public/indicador_por_pais_nacional/{pais}_nacion.csv'
    write_dataframe_to_csv_on_s3(table_piv, 'iadbprod-csd-hub-analyticaldata', ruta2)  

Pais: DO


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 2848 records to graphdata-mobility-public/indicador_por_pais_nacional/DO_nacion_analisis.csv
S3
(2848, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_nacional/DO_nacion.csv
S3
(1, 87)
Pais: SV


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 1246 records to graphdata-mobility-public/indicador_por_pais_nacional/SV_nacion_analisis.csv
S3
(1246, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_nacional/SV_nacion.csv
S3
(1, 87)
Pais: CR


...

Writing 2847 records to graphdata-mobility-public/indicador_por_pais_nacional/CO_nacion_analisis.csv
S3
(2847, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_nacional/CO_nacion.csv
S3
(1, 87)
Pais: BR


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 2403 records to graphdata-mobility-public/indicador_por_pais_nacional/BR_nacion_analisis.csv
S3
(2403, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_nacional/BR_nacion.csv
S3
(1, 87)
Pais: MX


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 2848 records to graphdata-mobility-public/indicador_por_pais_nacional/MX_nacion_analisis.csv
S3
(2848, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_nacional/MX_nacion.csv
S3
(1, 87)


### Indices a nivel regional

In [17]:
### Elimar folder "results" en instancia con terminal
#cd SageMaker/
#cd Fase2/1_Series\ de\ movilidad/
#rm -rf "results"

In [18]:
paises = ['AR','CO','PE','NI','SV','BO','DO','CL','CR','PY','BZ','HN','JM','TT','UY','GY','EC','GT','PA','VE','MX','BR']
#paises = ['CL']
for pais in paises:
    print(f"Pais: {pais}")
    
    df_by_day=df_by_day2at[df_by_day2at['iso_country_code']==pais]
    df_by_day=df_by_day[list(df_by_day.columns[:-1])]
    df_by_day=df_by_day.sort_values(by=['Mes','Dia','id_provincia'])
    
    ##computar indice suavizado
    ##Baseline    
    year="2020"
    df_by_day["fecha"] = year + "-" + df_by_day['Mes'].astype(int).astype(str) + "-" + df_by_day["Dia"].astype(int).astype(str)
    df_by_day['weekday'] = pd.to_datetime(df_by_day['fecha']).apply(lambda x: x.weekday())
    df_by_day["weekday2"]=0 
    df_by_day["weekday2"][df_by_day["weekday"]==6]="Domingo"       
    df_by_day["weekday2"][df_by_day["weekday"]==5]="Sabado" 
    df_by_day["weekday2"][(df_by_day["weekday"]!=5) & (df_by_day["weekday"]!=6)]="Semana" 
    df_by_day["key"]=df_by_day["weekday2"].astype(str)+df_by_day["id_provincia"].astype(str)
    reg=df_by_day["id_provincia"].unique()
    dia8=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==8)].groupby(["id_provincia"])["%MasDe1km_new"].sum()
    dia8=pd.DataFrame(dia8)
    dia8["fecha"]="Domingo"
    
    #dia8["id_provincia"]=reg
    dia8=dia8.reset_index()
    dia7=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==7)].groupby(["id_provincia"])["%MasDe1km_new"].sum()
    dia7=pd.DataFrame(dia7)
    dia7["fecha"]="Sabado"
    
    #dia7["id_provincia"]=reg
    dia7=dia7.reset_index()
    diasem=df_by_day[(df_by_day["Mes"]==2) & (df_by_day['Dia'].isin(['5', '6', '9', '10', '11']))].groupby(["id_provincia"])["%MasDe1km_new"].mean()
    diasem=pd.DataFrame(diasem)
    diasem["fecha"]="Semana"
    
    #diasem["id_provincia"]=reg
    diasem=diasem.reset_index()
    match=diasem.append(dia8)
    match=match.append(dia7)
    match["key"]=match["fecha"].astype(str)+match["id_provincia"].astype(str)
    match=match.rename(columns={"%MasDe1km_new": "Baseline"})
    match=match.loc[:,["key","Baseline"]]
    df_by_day=df_by_day.merge(match,how='left',on=['key'])
    
    ##Ratio
    df_by_day["ratio"]=df_by_day['%MasDe1km_new']/df_by_day['Baseline']
    
    ##por excepcion del día conflictivo
    df_by_day["ratio_lag1"]=df_by_day.groupby(["id_provincia"])["ratio"].shift(1)
    df_by_day["ratio_f1"]=df_by_day.groupby(["id_provincia"])["ratio"].shift(-1)
    
    ##Suavizado
    df_by_day["ratio_sua"]=(df_by_day["ratio"]+df_by_day["ratio_lag1"]+df_by_day["ratio_f1"])/3    
    
    ##Indice
    df_by_day["FracMas1km"]=df_by_day["ratio_sua"]-1
    
    ##key para tabla pivot
    df_by_day["Dia"]=df_by_day["Dia"].astype(int)
    df_by_day["Dia"]=df_by_day.Dia.map("{:02}".format)
    df_by_day["key4"]=df_by_day['Mes'].astype(int).astype(str)+ df_by_day["Dia"].astype(str)
    
    ##genero dataframe para exportar
    df_by_day2=df_by_day.loc[:,["Mes", "Dia","key4", "id_provincia", "MasDe1km", "n_users",'%MasDe1km_new',"FracMas1km"]]

    ##tabla pivot para publicar
    table_piv = pd.pivot_table(df_by_day, values='FracMas1km', index=['id_provincia'],
                    columns=['key4'], aggfunc=np.sum)
    table_piv=table_piv[table_piv.columns[1:-1]]
    table_piv = table_piv*100
    table_piv = table_piv.round(2)
    table_piv.index.name="cod"
    names_cols = []
    for jj in range(2,table_piv.shape[1]+2):
        names_cols.append(jj)
    table_piv.columns=names_cols
    
    ##exportando datos a S3
    ruta=f'graphdata-mobility-public/indicador_por_pais_mapa/{pais}_mapa.csv'
    write_dataframe_to_csv_on_s3(table_piv, 'iadbprod-csd-hub-analyticaldata', ruta)
    
    ruta2=f'graphdata-mobility-public/indicador_por_pais_analisis/{pais}_analsis.xlsx'
    write_dataframe_to_excel_on_s3(df_by_day2, 'iadbprod-csd-hub-analyticaldata', ruta2)
    
    ## guardando datos en la instancia - mapa
    if not os.path.exists(f'results'):  
        os.mkdir(f'results')
    if not os.path.exists(f'results/indicador_por_pais_mapa'):  
        os.mkdir(f'results/indicador_por_pais_mapa')
    if not os.path.exists(f'results_map/indicador_por_pais_mapa/{pais}'):  
        os.mkdir(f'results/indicador_por_pais_mapa/{pais}')        
    table_piv.to_csv(f'results/indicador_por_pais_mapa/{pais}/movilidad.csv',sep = ';')
    
    ## guardando datos en la instancia - analisis
    if not os.path.exists(f'results'):  
        os.mkdir(f'results')
    if not os.path.exists(f'results/indicador_por_pais_analisis'):  
        os.mkdir(f'results/indicador_por_pais_analisis')
    if not os.path.exists(f'results/indicador_por_pais_analisis/{pais}'):  
        os.mkdir(f'results/indicador_por_pais_analisis/{pais}')        
    df_by_day2.to_csv(f'results/indicador_por_pais_analisis/{pais}/analisis.csv',sep = ';')

Pais: AR


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 24 records to graphdata-mobility-public/indicador_por_pais_mapa/AR_mapa.csv
S3
(24, 87)
Pais: CO
Writing 32 records to graphdata-mobility-public/indicador_por_pais_mapa/CO_mapa.csv
S3
(32, 87)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Pais: PE
Writing 26 records to graphdata-mobility-public/indicador_por_pais_mapa/PE_mapa.csv
S3
(26, 87)


...

Writing 25 records to graphdata-mobility-public/indicador_por_pais_mapa/VE_mapa.csv
S3
(25, 87)
Pais: MX


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 32 records to graphdata-mobility-public/indicador_por_pais_mapa/MX_mapa.csv
S3
(32, 87)
Pais: BR


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 27 records to graphdata-mobility-public/indicador_por_pais_mapa/BR_mapa.csv
S3
(27, 87)


In [19]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for folder in os.listdir('results/indicador_por_pais_mapa'):
        for file in os.listdir(os.path.join('results/indicador_por_pais_mapa',folder)):
            ziph.write(os.path.join('results/indicador_por_pais_mapa', folder, file))

if __name__ == '__main__':
    zipf = zipfile.ZipFile(f'Indicador_mapa.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('results', zipf)
    zipf.close()
    
write_file_to_s3('iadbprod-csd-hub-analyticaldata',f'Indicador_mapa.zip',f'graphdata-mobility-public/indicador_por_pais_final/Indicador_mapa.zip')

In [20]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for folder in os.listdir('results/indicador_por_pais_analisis'):
        for file in os.listdir(os.path.join('results/indicador_por_pais_analisis',folder)):
            ziph.write(os.path.join('results/indicador_por_pais_analisis', folder, file))

if __name__ == '__main__':
    zipf = zipfile.ZipFile(f'Indicador_analisis.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('results', zipf)
    zipf.close()
    
write_file_to_s3('iadbprod-csd-hub-analyticaldata',f'Indicador_analisis.zip',f'graphdata-mobility-public/indicador_por_pais_final/Indicador_analisis.zip')